## 데이터 출처

[Naver sentiment movie corpus]: https://github.com/e9t/nsmc/

- CNN 모델의 학습을 위해 [Naver sentiment movie corpus] 데이터셋 중 10,000건을 추출하여 사용하였습니다.

In [1]:
# torchtext.legacy를 사용할 수 있는 torchtext 버전 설치
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 9.6 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [2]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch
# torch.nn : 신경망 구현을 위한 데이터 구조, 신경망 레이어, 관련함수들이 구현되어 있는 팩키지
# torch.nn.functional: torch.nn 팩키지의 함수들이 정의되어 있음 (손실함수, 활성화함수, 풀링함수 등) 
# torch.autograd : 미분을 위한 함수들이 정의되어 있음
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


# torchtext.legacy : text를 위한 데이터 로더를 제공, 다음과 같은 과정을 편리하게 지원
# 토크나이징(Tokenization)
# 단어장 생성(Build Vocabulary)
# 토큰의 수치화(Numericalize all tokens)
# 데이터 로더 생성(Create Data Loader)

from torchtext.legacy import data
import torchtext.datasets as datasets
import pickle

In [5]:
# CNN 모델의 구조와 연산을 정의
class CNN_Text(nn.Module):
    # 생성자 : 모델의 구조와 동작을 정의
    # 객체가 갖는 속성값을 초기화함. 객체가 생성될 때 자동으로 호출된다.
    def __init__(self, embed_num, class_num):
        super(CNN_Text, self).__init__() # nn.Module 클래스를 초기화
        # V: 사전의 크기
        # D: embed_dim
        # C: 분류하고자 하는 클래스의 개수
        # Co : 각 커널(필터)의 갯수
        V = embed_num
        D = 100 
        C = class_num
        Co = 50         # output channel 수 (필터의 갯수)
        Ks = [2,3,4]

        # 사전에 있는 모든 단어 벡터에 random 초기값
        self.embed = nn.Embedding(V, D) # 예) 1000*100
        # torch.nn.Conv2d (in_channels, out_channels, kernel_size, stride=1)
        # convs1에 컨볼루션 모듈의 리스트가 들어감 (필터(커널) 갯수만큼) 
        # forward에서 순차적으로 접근 가능
        self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, (K, 100)) for K in Ks])
        self.dropout = nn.Dropout(0.2)

        # nn.Linear 클래스. Fully Connected Layer 
        # Applies a linear transformation to the incoming data (y = Wx + b)
        # torch.nn.Linear(in_features, out_features)
        # in_features: size of each input sample, out_features: size of each output sample 
        self.fc1 = nn.Linear(len(Ks)*Co, C) # 3(kernel)*50(output)*2(긍정, 부정)

        # foward 함수 : 모델이 학습데이터를 입력받아서 forward 연산을 진행
        # model 객체를 데이터와 함께 호출하면 자동으로 실행된다.
    def forward(self, x):
        x = self.embed(x)   # (N, W, D) 미니배치(N), 문장 최대길이, 단어벡터 차원
        x = x.unsqueeze(1)  # (N x Ci x W x D) Conv2d를 사용하려면 ** 입력채널 수 추가 **해야 함

        # Convolution Layer
        # Convolution -> ReLU -> 텐서의 dimension 3을 squeeze(max_pool1d는 3D 입력을 받음)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]

        # Max Pooling
        # F.max_pool1d(input, kernel_size): Applies a 1D max pooling over an input
        # Tensor.size(dim=None) : Returns the size of the self tensor. If dim is specified, returns the size of that dimension.
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks) max pooling 후에 마지막 차원은 1 -> squeeze
        x = torch.cat(x, 1) # torch.cat(tensors, dim), dim=1이면 두번째 차원이 늘어나게 concat (첫번째 차원은 N)
        x = self.dropout(x) # (N, len(Ks)*Co), dropout을 적용
        logit = self.fc1(x) # fully-connected layer 적용
        return logit

In [6]:
class mydataset(data.Dataset):
    @staticmethod  # 유틸리티 메소드 정의 시 선언
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)] # text_field 레이블은 text, label_field 레이블은 label
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')               
                                  
                examples += [ data.Example.fromlist( [ txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [11]:
# Field : 텐서로 변환될 텍스트 데이터타입을 정의 
# text_field, label_field : 전처리 관련된 field 객체를 각각 생성 
# batch_first : 미니배치 차원을 맨 앞에 둔 텐서를 생성할 것인지
# fix_length : 하나의 문장 내 max 토큰수 
# sequential : 시퀀스 데이터 여부
text_field = data.Field(batch_first = True, fix_length = 20) # dimension 0에 있는 파라미터는 배치 차원이다
label_field = data.Field(sequential= False, batch_first = True, unk_token = None) # unk_token을 표현할 스트링

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_train_tok.txt') # text와 label을 결합을 해서 하나의 객체로 만들어줌

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/aivle/data/nsm/small_ratings_test_tok.txt')
#print(test_data.fields.items())

# vocab 생성
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

# Data Loader 생성 (train_data, test_data를 각각 100개, 1개씩 데이터 로딩)
train_iter, test_iter = data.Iterator.splits(   # train과 test를 넘겨줘서 iterator를 train 100개 test는 1개로 반복적으로 로딩
                            (train_data, test_data), 
                            batch_sizes=(100, 1))#, device= 'cuda')
len(text_field.vocab)

21893

In [14]:
# CNN모델 객체를 생성 (embed_num, class_num)
cnn = CNN_Text(len(text_field.vocab),2)

# torch.optim : 신경망 학습을 위한 다양한 파라미터 최적화 알고리즘이 구현되어 있는 팩키지
# Optimizer를 설정
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train() # start할 준비를 끝마침 -> train mode를 시작하겠다 (내부 구조를 찍어서 확인 가능)


CNN_Text(
  (embed): Embedding(21893, 100)
  (convs1): ModuleList(
    (0): Conv2d(1, 50, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 50, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 50, kernel_size=(4, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=150, out_features=2, bias=True)
)

In [ ]:
# sqeeze(index) => sqeeze index에 3을 주면 3차원으로 줄어든다
# 즉 output에 차원을 맞춰주기 위해서 등 사용
# unsqeeze는 채널을 하나 추가하기 위해서 차원을 하나 추가하기 위해서
# tensor의 형태를 맞추기 위해서 호출하는 함수

In [16]:
for epoch in range(20):
    
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad() # resets the gradient to 0
        
        txt = batch.text
        label = batch.label
                
        #print(txt.size()) -> torch.Size([100, 20])
        pred = cnn(txt)
                
        #print(pred.size(), label.size()) -> torch.Size([100, 2]) torch.Size([100])
        #print(label)
        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward() # backward 연산
        optimizer.step() # 파라미터 업데이트
        
    print(epoch,'epoch')    
    print('loss : {:.3f}'.format(totalloss.numpy()))

torch.save(cnn,'/content/gdrive/My Drive/Colab Notebooks/aivle/model/cnn_model.pt')

0 epoch
loss : 1.833
1 epoch
loss : 1.498
2 epoch
loss : 1.722
3 epoch
loss : 1.364
4 epoch
loss : 1.325
5 epoch
loss : 1.182
6 epoch
loss : 1.136
7 epoch
loss : 1.110
8 epoch
loss : 1.047
9 epoch
loss : 1.096
10 epoch
loss : 0.978
11 epoch
loss : 1.004
12 epoch
loss : 0.935
13 epoch
loss : 1.094
14 epoch
loss : 0.905
15 epoch
loss : 0.900
16 epoch
loss : 0.956
17 epoch
loss : 0.824
18 epoch
loss : 0.888
19 epoch
loss : 0.942


In [18]:
%%time
from sklearn.metrics import classification_report
cnn.eval() # 모델을 evaluation mode로 설정. 정규화 기술(dropout 등)을 배제하여 온전한 모델로 평가
correct = 0
incorrect = 0
y_test = []
prediction = []

for batch in test_iter:
    txt = batch.text
    label = batch.label
    y_test.append(label.data[0]) #  긍/부정

    pred = cnn(txt)
    _,ans = torch.max(pred,dim=1) # dimension을 기준으로 (최대값, 최대값이 있는 인덱스) 반환 
    prediction.append(ans.data[0]) # ans.data[0]: 최대값이 들어있는 인덱스 (0 또는 1) 
  
    if ans.data[0] == label.data[0]:        
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print(classification_report(torch.tensor(y_test),     # 정답값
                            torch.tensor(prediction), # 예측값
                            digits=4,                 # 출력할 자리수
                            target_names=['negative', 'positive'])) # display names matching the label

# Weighted Avg는 클래스의 수치간의 평균 

correct :  78
incorrect :  22
              precision    recall  f1-score   support

    negative     0.8889    0.6400    0.7442        50
    positive     0.7188    0.9200    0.8070        50

    accuracy                         0.7800       100
   macro avg     0.8038    0.7800    0.7756       100
weighted avg     0.8038    0.7800    0.7756       100

CPU times: user 71.7 ms, sys: 26 µs, total: 71.7 ms
Wall time: 72.5 ms
